# Assignment 3 – Topic Modeling and Clustering for Online Social Media Data

*Due: Friday January 12 at 14:00 CET*

In the third assignment of the course Applications of Machine Learning (INFOB3APML), you will learn to use topic modeling and clustering to identify topics in online social media data. The objectives of this assignment are:
- understand and process the text data
- use the clustering algorithm to determine clusters in real-life data
- use the Latent Dirichlet Allocation algorithm to identify discussed topics in real-life text data 
- use the visualization tools to validate the results of unsupervised learning and interpret your findings
- reflect on the difference between two type of unsupervised learning algorithms

In this assignment, you are going to discover the different ‘topics’ from a real social media text dataset. The project is divided into two parts (4 subtasks):

- The first part contains data processing (1.1) and feature extraction (1.2) from the raw text data.
- In the second part, you will implement two methods (2.1), a topic modeling method and a clustering method, to identify topics from the processed data. Then, the evaluation will be done by using visualization tools (2.2). 

Provided files:
- The dataset: data/raw_data.txt
- A tutorial notebook showcases some packages you could use for this assignment (optional): Ass3_tutorial.ipynb
- Some sample visualization codes for interpreting the topic results: viz_example.ipynb

In [ ]:
import io
import spacy
import re
from tqdm import tqdm
import pickle
from PIL import Image
import numpy as np
from emoji import demojize
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, Birch
from sklearn.decomposition import PCA
from collections import Counter
import gensim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from pprint import pprint
from wordcloud import WordCloud
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

 ## Dataset:
 The data used in this assignment is Dutch text data. We collected the COVID-19 crisis related messages from online social media (Twitter) from January to November 2021. Then, a subset of raw tweets was randomly sampled. In total, our dataset includes the text data of about 100K messages. **To protect the data privacy, please only use this dataset within the course.**

In [ ]:
# read the data

def phase0_open_txt_stream(filename):
    return io.open(filename, "r", encoding="utf-8")


pipe = phase0_open_txt_stream("others/data/raw_data.txt")

all_messages = []

for message in pipe:
    all_messages.append(message.strip())

pipe.close()

In [ ]:
print(f"{len(all_messages)} messages")

 ## 0. Before you start the Project: 
 The provided messages in the raw dataset were collected based on 10 different themes that relate to the COVID-19 crisis. Here is a list of all themes:
 -	Lockdown
 -	Face mask
 -	Social distancing
 -	Loneliness
 -	Happiness
 -	Vaccine
 -	Testing
 -  Curfew
 -  Covid entry pass
 -  Work from home

Before starting your project, you need to first filter the messages (all messages are in Dutch) and use the messages belonging to only one theme for the topic identification. 
 
If you have submitted the theme preference, you can skip the following paragraph.

*Please notice that there will be maximum two teams working on a same theme. In this way, we hope that each group will develop their own dataset and come up with interesting results.*

 ## 1.1 Data Processing
 In the first part of the assignment, please first filter the messages and use the messages belonging to your allocated theme for the identification of topics. For that you will need to:
 -	Design your query (e.g. a regular expression or a set of keywords) and filter the related messages for your allocated theme. 
 -	Clean your filtered messages and preprocess them into the right representation. Please refer to the text data pre-processing and representation methods discussed in the lecture. You may use some of the recommended packages for text data preprocessing and representation.

In [ ]:
# TODO: filter the related messages
facemask_messages = []

facemask_pattern = r"mondkap|(mond)?masker|bekluier|beklap|\bmask|muilkorf|facemask|face mask|monddoek|face shield|gelaats?scherm|gezichtsmasker|muilmasker|gezichtsbeschermingskapje|gezichtsschild|kap|eendenbek|gelaatsmasker"
for message in all_messages:
    if re.search(facemask_pattern, message.lower()):
        facemask_messages.append(message)

In [ ]:
print(f"{len(facemask_messages)} messages about facemasks")

In [ ]:
# TODO: clean and preprocess the messages

def lowercase_doc(doc: str) -> str:
    """
    Transform a document into lowercase.
    """
    return doc.lower()


def remove_urls(doc: str) -> str:
    """
    Return a document stripped of any URLs.
    """
    url_pattern = r"http[s]*\S+"
    clean_doc = re.sub(url_pattern, ' ', doc)
    return clean_doc


def remove_punctuation(doc: str) -> str: 
    """
    Transform a text document into a document without punctuation, digits, newline characters and twitter tags (@HugodeJonge, etc.).
    """
    doc = doc.replace('\\n', ' ')

    punct_pattern = r"@\w+\b|[^\w\s]|\d+|\brt\b"
    
    clean_doc = re.sub(punct_pattern, ' ', doc, )

    return clean_doc
    

def tokenize(doc: str, nlp, lemma = True) -> list:
    """
    Transform string document into a tokenized list.
    The lemma parameter specifiec whether to return lemmatized words or not.
    """
    if lemma:
        return [token.lemma_ for token in nlp(doc) if not token.is_stop]
    else:
        return [token for token in nlp(doc) if not token.is_stop]

def convert_mondkapje_to_mondkap(doc: str) -> str:
    """
    Convert every occurence of 'mondkapje' into 'mondkap'.
    """
    
    doc = doc.replace('mondkapje', 'mondkap')

    return doc

In [ ]:
nlp = spacy.load("nl_core_news_sm")

cleaned_messages = []

for sentence in tqdm(facemask_messages):

    sentence = lowercase_doc(sentence)
    sentence = demojize(sentence)
    sentence = remove_urls(sentence)
    sentence = remove_punctuation(sentence)
    tokenized = tokenize(sentence, nlp)
    cleaned_message = ' '.join(tokenized)
    cleaned_message = convert_mondkapje_to_mondkap(cleaned_message)

    cleaned_messages.append(cleaned_message)

In [ ]:
for before, after in zip(facemask_messages[0:5], cleaned_messages[0:5]):
    print(f"Before cleaning:\n{before}\nAfter cleaning:\n{after}\n")

In [ ]:
# TODO: represent the messages into formats that can be used in clustering or LDA algorithms (you may need different represention for two algorithms)

# LDA input
vectorizer = CountVectorizer(min_df = 1)
lda_input = vectorizer.fit_transform(cleaned_messages)
lda_featurenames = vectorizer.get_feature_names_out()

In [ ]:
# Clustering input
vectorizer = TfidfVectorizer()
vectorized = vectorizer.fit_transform(cleaned_messages)
clustering_input = vectorized.toarray()
clustering_featurename = vectorizer.get_feature_names_out()


 ## 1.2 Exploratory Data Analysis
 After preprocessing the data, create at least 2 figures or tables that help you understand the data.

 While exploring the data, you may also think about questions such as:
 - Can you spot any differences between Twitter data and usual text data?
 - Does your exploration reveal some issues that would make it difficult to interpret the topics?
 - Can you improve the data by adding additional preprocessing steps?

In [ ]:
# TODO: plot figure(s)

# WORDCLOUD

facemask_image = np.array(Image.open("mondkapplaatje.jpg"))

wordcloud_messages = ' '.join(cleaned_messages)
wordcloud = WordCloud(width=800, height=400, background_color='white', mask = facemask_image).generate(wordcloud_messages)

plt.figure(figsize=(50, 30))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
hashtags = []

for tweet in facemask_messages:
    hashtags += re.findall(r'#(\w+)', tweet)
    
hashtag_frequence = Counter(hashtags)

hashtags_sorted = hashtag_frequence.most_common(5)
hashtag_x, hashtag_y = zip(*hashtags_sorted)

plt.figure(figsize=[10, 10])
plt.bar(hashtag_x, hashtag_y, color='darkslategrey')
plt.xlabel('Most occurring hashtags')
plt.ylabel('Frequency per hashtag')
plt.title('Top 5 most commonly used hashtags')
plt.show()

## 2.1 Topic modelling and clustering
 In the second part of the assignment, you will first:
 -	Implement a Latent Dirichlet Allocation (LDA) algorithm to identify the discussed topics for your theme
 -	Implement a clustering method  to cluster messages into different groups, then represent the topic of each cluster using a bag of words

While implementing the algorithms, you may use the codes from the recommended packages. In the final report, please explain reasons to select the used algorithm/package. 

In [ ]:
# GENSIM
words = [message.split() for message in cleaned_messages]
id2word = gensim.corpora.Dictionary(words)
corpus = [id2word.doc2bow(word) for word in words]

gensim_lda = gensim.models.LdaMulticore(
    num_topics = 5,
    corpus = corpus,
    id2word = id2word
)

gensim_lda.print_topics()

In [ ]:
# TODO: cluster the messages using a clustering algorithm
# TODO: cluster the messages using a clustering algorithm
k = 6

# Principal Component Analysis for a better visualization
pca = PCA(n_components = 2)
visualization_data = pca.fit_transform(clustering_input)

kmeans = KMeans(n_clusters = k, random_state = 9)
kmeans.fit(clustering_input)
labels = kmeans.labels_

plt.figure()

for i in range(k):
    plt.scatter(
        visualization_data[labels == i, 0],
        visualization_data[labels == i, 1]
    )
    
plt.title("Text clustering")
plt.show()


In [ ]:
words_per_cluster = []

for label in range(k):
    indices = np.where(labels == label)[0]

    cluster = np.array(cleaned_messages)[indices]

    all_cluster_words = []
    for sentence in cluster:
        all_cluster_words += sentence.split()
    
    words_per_cluster.append(all_cluster_words)

words_per_cluster

In [ ]:
for cluster_words in words_per_cluster:

    wordcloud_messages = ' '.join(cluster_words)
    wordcloud = WordCloud(width=600, height=600, background_color='white').generate(wordcloud_messages)

    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

 ## 2.2 Results, evaluation and Interpretation 
 
Finally, you will describe, evaluate and interpret your findings from two methods. 

- In the report, you need to describe and discuss the similarity and difference of results from two methods.
- While evaluating the results, human judgment is very important, so visualization techniques are helpful to evaluate the identified topics in an interpreted manner. 
    
1. For evaluating the topic modelling algorithm, please first use the interactive tool **[pyLDAvis](https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb#topic=0&lambda=1&term=)** to examine the inter-topic separation of your findings. 

2. For interpreting the identified topics / clusters of both algorithms, we provide example code for several visualization techiques. You can use multiple ones to evaluate your results or come up with visualisations on your own. The files contain examples for how to use the visualisation functions.


In [ ]:
# TODO: evaluation 
# Transform the gensim LDA model for visualization
lda_display = gensimvis.prepare(gensim_lda, corpus, id2word, sort_topics=False)

# Display the visualization
pyLDAvis.display(lda_display)


# Bonus Tasks 

We would like to challenge you with the following bonus task. For each task that is successfully completed, you may obtain max. 1 extra point. 

1. Implement another clustering algorithm or design your own clustering algorithm. Discuss your findings and explain why this is a better (or worse) clustering algorithm than the above one (the clustering algorithm, not LDA).

2. Can you think of other evaluation methods than the provided visualization techniques? If so, implement one and explain why it is a good evaluation for our task.

In [ ]:
# ANOTHER CLUSTERING ALGORITHM

# TODO: cluster the messages using a clustering algorithm
k = 6

# Principal Component Analysis for a better visualization
pca = PCA(n_components = 2)
visualization_data = pca.fit_transform(clustering_input)

kmeans = Birch(n_clusters = 6)
kmeans.fit(clustering_input)
labels = kmeans.labels_

plt.figure()

for i in range(k):
    plt.scatter(
        visualization_data[labels == i, 0],
        visualization_data[labels == i, 1]
    )
    
plt.title("Text clustering")
plt.show()

In [ ]:
words_per_cluster = []

for label in range(k):
    indices = np.where(labels == label)[0]

    cluster = np.array(cleaned_messages)[indices]

    all_cluster_words = []
    for sentence in cluster:
        all_cluster_words += sentence.split()
    
    words_per_cluster.append(all_cluster_words)

words_per_cluster

In [ ]:
for cluster_words in words_per_cluster:

    wordcloud_messages = ' '.join(cluster_words)
    wordcloud = WordCloud(width=600, height=600, background_color='white').generate(wordcloud_messages)

    plt.figure(figsize=(10, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()